In [ ]:
import numpy as np
import random
import math
import networkx as nx
import matplotlib.pyplot as plt
import os

In [ ]:
%run ./Generate_Network_Topology.ipynb

In [ ]:
%run ./Generating_CRs.ipynb 

In [ ]:
%run ./Links.ipynb
# Major issue : Number of links

In [ ]:
# Define the subdirectory name
subdirectory = "Figures"

# Check if the subdirectory exists
if not os.path.exists(subdirectory):
    os.makedirs(subdirectory)


In [ ]:
# 21 Edges
edges = [ (1, 2, 1100),  #Edge between Node 0 and Node 1
             (2, 3, 1600), #Intermediate Edge between Node 1 and Node 2
             (1, 3, 600), # Edge between Node 0 and Node 2
             (2, 8, 1500), # Edge between Node 1 and Node 7
             (1, 4, 1000),  # Edge between Node 0 and Node 3
             
         (4, 5, 600), # Edge between Node 3 and Node 4
             (5, 7, 800), # Intermediate edge between Node 4 and Node 6
             (7, 8, 700), # Intermediate edge between Node 6 and Node 7
             (10, 11, 900), # Edge Betweeen Node 9 and Node 10
             (4, 9, 1500), # Edge Betweeen Node 3 and Node 8
         
         (9, 12, 800), # Edge Betweeen Node 8 and Node 11
             (8, 10, 700), # Edge Betweeen Node 7 and Node 9
             (10, 12, 500), # Edge Betweeen Node 9 and Node 11
             (9, 14, 800), # Edge Betweeen Node 8 and Node 13
             (10, 14, 500), # Edge Betweeen Node 9 and Node 13
         
         (12, 13, 300), # Edge Betweeen Node 11 and Node 12
             (13, 14, 300), # Edge Betweeen Node 12 and Node 13
             (3, 6, 1000), # Edge Betweeen Node 2 and Node 5
             (6, 5, 1100), # Edge Betweeen Node 5 and Node 4
             (6, 13, 2000), # Edge Betweeen Node 5 and Node 12
         
         (6, 11, 1200), # Edge Betweeen Node 5 and Node 10
        ]

# Step 1:
Initialize the network status and parameters, and generate a set of CR, i.e., X CRs.
## Step 2(a) : 
For each CR, we set a SL (incorporated in Step 1)

In [ ]:
g_nsfnet = create_topology(edges) # Creates the nsfnet topology
links, indices, unordered_indices = Links.initialize_links(edges) # Initializes the links structure with weights

numNodes = len(g_nsfnet)
X = 100    # Just an example
CRs = CR.generate_crs(X) # Generating X Crs. 

In [ ]:
# Print the array of CR instances
print("CR info : ")
for cr in CRs:
    cr.display_info()

# Print link info
'''
for link in links[indices]:    # Will display double the number of actual edges, due to ordered pair
   link.display_info()
'''
print("Link info : ")
Links.display_all_links()

# Step 2(b) :
use the K-SP algorithm to obtain the K candidate transmission paths. 
The path with the shortest distance is selected as the transmission path of QSC and PIC.

In [ ]:
cr = CRs[0]    # Taking only 1 cr for now. Later it can be converted to for loop to loop over all CRs

k_sp = cr.k_sp(g_nsfnet, 10)    # Taking k = 10
candidate_path = k_sp[0][0]    # Path with the shortest distance becomes the transmission path

In [ ]:
# Displaying all the K shortest paths along with their path costs
for i, (path, cost) in enumerate(k_sp):
    print(f"Path {i + 1}: {path}, Cost: {cost}")

# Displaying the candidate path
print(f"Candidate path : {candidate_path}")

In [ ]:
# Allocate to TDC and QSC

In [ ]:
# if no time slot is available, we move on to the next candidate path
cr = CRs[0]
attempt = Links.FF(candidate_path, cr)
print(f"CR status : {cr.status}, on path : {candidate_path}, allocated resources : {cr.allocated_resources}")

# Step3：
If a CR is successfully allocated to TDC & QSC, turn to Step4; otherwise, this CR blocks. If an event is waiting for a CR(X=0), turn to Step9


In [ ]:
if attempt:
    print(f"CR status : {cr.status}, on path : {candidate_path}, allocated resources : {cr.allocated_resources}" )
else :
    print("CR couldn't be successfully allocated on path :", candidate_path)

In [ ]:
# Displaying the resources after allocation
path = candidate_path
for s, d in zip(path, path[1:]):
    links[s, d].display_info()
    #links[s, d].wl_info()

# Step 4 :
Create three queues with different priorities in PQ={𝑪𝑹1, 𝑪𝑹0, 𝑪𝑹−1}, i.e., 𝑪𝑹1, 𝑪𝑹0 and 𝑪𝑹−1. Each CR has a specific SL, sl. 
Place the CR of the corresponding SL in the corresponding priority. If this CR belongs to the high-level priority queue, 𝑪𝑹1, then turn to Step 5. 
If this CR belongs to mid-level priority queue 𝑪𝑹0 or low-level priority queue, 𝑪𝑹−1, turn to Step 6.


In [ ]:
PQ = CR.create_priority_queue()

In [ ]:
#CR.display_all(PQ)

# Trying Manually

In [ ]:
'''
Let's assume we have 800 CRs, all dealt with.
Manually calculating TUR : calculate total number of used slots. divided by total slots(already known)
2 approaches : 
CR approach : for every allocated CR, we take the number of links in its path = len(path)-1. This will be the occupied slots by a particular cr. Can calculate SRCR and NSP as well
Links approach : count the nonzero elements in each link(unordered_indices) band.
'''

In [ ]:
X = 600
# Parameters used later
k = 10
   
# Initializes the links structure with weights
(links, indices, unordered_indices) = Links.initialize_links(edges) 

# Generating X Crs
CRs = CR.generate_crs(X) 

########################################################################
    
# Creating the priority Queue
PQ = CR.create_priority_queue()

for cr in PQ:  
            
    k_sp = cr.k_sp(g_nsfnet, k)    # Taking k = 10. Gives us [path][weight]
    # Finding the path to be followed
    for i in range(k):    # Using the next k-sp in case the CR couldn't be allocated to the previous one
        candidate_path = k_sp[i][0]    # Path with the lowest weight so far becomes the transmission path
        print(f"\nAttempting allocation of CR {cr.index} on path : {candidate_path}")
        
        # Attempting to allocate resources
        success = Links.FF(candidate_path, cr)
        print(f"CR {cr.index} status : {cr.status}, on path :", candidate_path)
        # Checking if the path was allocated or not
        if success:
            break  

        #X = X-1

In [ ]:
# CR approach(cra) : for tur, nsp and srcr

#X = 800
"""
cra_tur = np.zeros(j_range).astype(int)
cra_srcr = np.zeros(j_range).astype(int)
cra_nsp = np.zeros(j_range).astype(int)
"""
cra_nsp = 0
cra_tur = 0
cra_srcr = 0
for i, cr in enumerate(PQ):    # PQ_all[1] is for crs with high priority
    if cr.status == 'allocated':
        cra_tur += (len(cr.path) - 1)
        cra_srcr += 1
        cra_nsp += CR.sfw[cr.sl]

cra_tur /= (Links.total_ts * len(edges))
cra_srcr /= len(CRs)

In [ ]:
print((Links.total_ts * len(edges)))

In [ ]:
print(cra_tur, cra_nsp, cra_srcr)

In [ ]:
# Links approach(la): Only for tur

#X = 800
la_tur1 = 0
la_tur = np.zeros(4)
for nodes in Links.unordered_indices:
    la_tur1 += (Links.links[nodes].occupied_ts[0])
    la_tur += Links.links[nodes].occupied_ts

la_tur1 /= Links.total_ts * len(edges)

print(la_tur1)

la_tur[0] = la_tur1
for i in range(1, 4):
    la_tur[i] /= Links.total_ts * Links.channel_ts[i]

print(la_tur)

In [ ]:
print(la_tur, la_tur1)

In [ ]:
#CR.display_all(PQ)

# Combined

## QKRA

Network topology can be initialized once at the start. Only Links need to be initialized everytime

In [ ]:
#%%capture
aslc = "ASSL"
# Modify : X, k, num_iter, j_range, factor, expression of X in QKRA.ipynb
filename = "QKRA-" + aslc + ".txt"
file = open(filename, "a")
# Creates the nsfnet topology
g_nsfnet = create_topology(edges) 
numNodes = len(g_nsfnet)
X = 10   

# Parameters used later
k = 10    

# Let's test for X = 10, and j < 6. Average over 10 iterations.
num_iter = 100
j_range = 15
srcr = np.zeros([num_iter, j_range])
tur = np.zeros([num_iter, j_range])
nsp = np.zeros([num_iter, j_range]).astype(int)
x = np.zeros([num_iter, j_range]).astype(int)

file.write("QKRA-" + aslc + " : " + "k = " + str(k) + ", Averaged over(z) = " + str(num_iter) + "\n")

#factor = int(input("Input the multiplication factor"))
factor = 1

for z in range(num_iter):    # Loop to average over the same list of X

    #X = int(input("Input the number of CRs to be generates"))
    #X = 60*factor
    
    j = 0    
    %run ./QKRA.ipynb
    print(f"Iteration {z+1} complete")

In [ ]:
print(srcr, tur, nsp, x)

In [ ]:
avg_srcr = np.zeros(j_range)
avg_tur = np.zeros(j_range)
avg_nsp = np.zeros(j_range)
avg_x = np.zeros(j_range)

for j in range((j_range)):
    for i in range(num_iter):
        avg_srcr[j] += srcr[i, j] 
        avg_tur[j] += tur[i, j] 
        avg_nsp[j] += nsp[i, j] 
        avg_x[j] += x[i, j] 

avg_srcr /= num_iter
avg_tur /= num_iter 
avg_nsp /= num_iter
avg_x /= num_iter 
file.write("X range : " + str(avg_x) + "\n Average SRCR : " + str(avg_srcr) + "\n Average TUR : " + str(avg_tur) + "\n" )
file.close()

In [ ]:
print(avg_srcr, avg_tur, avg_nsp, avg_x)

In [ ]:
print(len(srcr), len(tur), len(nsp), len(x))

# Plotting the data

**SRCR and TUR**

In [ ]:
#y = [item[0] for item in tur]
plt.scatter(avg_x, avg_srcr, label='SRCR - scatter')
plt.scatter(avg_x, avg_tur, label='TUR - scatter')

plt.plot(avg_x, avg_srcr, label='SRCR - line')
plt.plot(avg_x, avg_tur, label='TUR - line')

plt.minorticks_on()
plt.xlabel('X = Number of CRs')
plt.ylabel('Ratio')
plt.title(f"Network Evaluation(averaged over {num_iter})")
plt.legend()
plt.show()

filename = "SRCR and TUR for X - " + str(avg_x[0]) + " to " + str(avg_x[-1]) + ".jpg"
print(filename)

# Combine the subdirectory and filename to create the full path
full_path = os.path.join(subdirectory, filename)

#plt.savefig(full_path, dpi=300, bbox_inches='tight')


**NSP**

In [ ]:
#y = [item[0] for item in nsp]
plt.scatter(avg_x, avg_nsp)
plt.plot(avg_x, avg_nsp)

plt.minorticks_on()
plt.xlabel('X = Number of CRs')
plt.ylabel('NSP')
plt.title(f"Network Evaluation(averaged over {num_iter})")
#plt.legend()
plt.show()

filename = "NSP for X - " + str(avg_x[0]) + " to " + str(avg_x[-1]) + ".jpeg"
print(filename)

# Combine the subdirectory and filename to create the full path
full_path = os.path.join(subdirectory, filename)

plt.savefig(full_path, dpi=300, bbox_inches='tight')


In [ ]:
for i in range(j_range):
    print(f"\nfor X = {5*(2 + i)*factor}")
    print(f"\nSRCR {i} : {srcr[i]}")
    print(f"\nNSP {i} : {nsp[i]}")
    print(f"\nTUR {i} : {tur[i]}")